## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#데이터있는 주소
%cd /content/drive/MyDrive/Ai/
!unzip -qq "/content/drive/MyDrive/Ai/open.zip"

Mounted at /content/drive
/content/drive/MyDrive/Ai
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
%pip install segmentation-models-pytorch
%pip install ttach

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.8 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16427 sha256=7e81569e682a1814e63cea06ad34a2da0f4b116faf84da5d05ee8d7115fd88e6
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60945 sha256=a9c7310c246fde83e8d3eb8f0eab5d3a4006beef7fd4683b160880fe6807fd32
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c6

In [ ]:
import os
import cv2
import random
import pandas as pd
import numpy as np
import ttach as tta
import math

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim.lr_scheduler import _LRScheduler

from sklearn.model_selection import KFold, train_test_split

from tqdm import tqdm

import albumentations as A
from albumentations.pytorch import ToTensorV2

from typing import List, Union
from joblib import Parallel, delayed

import segmentation_models_pytorch as smp
import argparse
import ssl

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Fix Randomseed

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(41) # Seed 고정

## Utils

In [ ]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## Data Load

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/Ai/train.csv')

## Custom Dataset

In [ ]:
class SatelliteDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_rle = self.data.iloc[idx, 2]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

In [ ]:
train_transform = A.Compose(
    [
        A.RandomCrop(224, 224, p=1),
        A.Normalize(),
        ToTensorV2()
    ]
)

val_transform = A.Compose(
    [
        A.CenterCrop(224, 224, p=1),
        A.Normalize(),
        ToTensorV2()
    ]
)

test_transform = A.Compose(
    [
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2()
    ]
)

## Data Loader

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

ENCODER = 'timm-efficientnet-b7'
ENCODER_WEIGHTS = 'advprop'
ACTIVATION = 'sigmoid'
DEVICE = 'cuda'

from segmentation_models_pytorch.encoders import get_preprocessing_fn
preprocess_input = get_preprocessing_fn('timm-efficientnet-b7', pretrained='advprop')

model = smp.UnetPlusPlus(
    encoder_name = ENCODER,
    encoder_depth=4,
    encoder_weights = ENCODER_WEIGHTS,
    in_channels = 3,
    decoder_channels=(256, 128, 64, 32),
    classes = 1,
    activation = ACTIVATION,
)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b7_ap-ddb28fec.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b7_ap-ddb28fec.pth
100%|██████████| 254M/254M [00:02<00:00, 124MB/s]


##Loss Function

In [ ]:
import re

class BaseObject(nn.Module):
    def __init__(self, name=None):
        super().__init__()
        self._name = name

    @property
    def __name__(self):
        if self._name is None:
            name = self.__class__.__name__
            s1 = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", name)
            return re.sub("([a-z0-9])([A-Z])", r"\1_\2", s1).lower()
        else:
            return self._name


class Metric(BaseObject):
    pass


class Loss(BaseObject):
    def __add__(self, other):
        if isinstance(other, Loss):
            return SumOfLosses(self, other)
        else:
            raise ValueError("Loss should be inherited from `Loss` class")

    def __radd__(self, other):
        return self.__add__(other)

    def __mul__(self, value):
        if isinstance(value, (int, float)):
            return MultipliedLoss(self, value)
        else:
            raise ValueError("Loss should be inherited from `BaseLoss` class")

    def __rmul__(self, other):
        return self.__mul__(other)


class SumOfLosses(Loss):
    def __init__(self, l1, l2):
        name = "{} + {}".format(l1.__name__, l2.__name__)
        super().__init__(name=name)
        self.l1 = l1
        self.l2 = l2

    def __call__(self, *inputs):
        return self.l1.forward(*inputs) + self.l2.forward(*inputs)


class MultipliedLoss(Loss):
    def __init__(self, loss, multiplier):

        # resolve name
        if len(loss.__name__.split("+")) > 1:
            name = "{} * ({})".format(multiplier, loss.__name__)
        else:
            name = "{} * {}".format(multiplier, loss.__name__)
        super().__init__(name=name)
        self.loss = loss
        self.multiplier = multiplier

    def __call__(self, *inputs):
        return self.multiplier * self.loss.forward(*inputs)

In [ ]:
def _take_channels(*xs, ignore_channels=None):
    if ignore_channels is None:
        return xs
    else:
        channels = [channel for channel in range(xs[0].shape[1]) if channel not in ignore_channels]
        xs = [torch.index_select(x, dim=1, index=torch.tensor(channels).to(x.device)) for x in xs]
        return xs


def _threshold(x, threshold=None):
    if threshold is not None:
        return (x > threshold).type(x.dtype)
    else:
        return x


def iou(pr, gt, eps=1e-7, threshold=None, ignore_channels=None):
    """Calculate Intersection over Union between ground truth and prediction
    Args:
        pr (torch.Tensor): predicted tensor
        gt (torch.Tensor):  ground truth tensor
        eps (float): epsilon to avoid zero division
        threshold: threshold for outputs binarization
    Returns:
        float: IoU (Jaccard) score
    """

    pr = _threshold(pr, threshold=threshold)
    pr, gt = _take_channels(pr, gt, ignore_channels=ignore_channels)

    intersection = torch.sum(gt * pr)
    union = torch.sum(gt) + torch.sum(pr) - intersection + eps
    return (intersection + eps) / union


jaccard = iou


def f_score(pr, gt, beta=1, eps=1e-7, threshold=None, ignore_channels=None):
    """Calculate F-score between ground truth and prediction
    Args:
        pr (torch.Tensor): predicted tensor
        gt (torch.Tensor):  ground truth tensor
        beta (float): positive constant
        eps (float): epsilon to avoid zero division
        threshold: threshold for outputs binarization
    Returns:
        float: F score
    """

    pr = _threshold(pr, threshold=threshold)
    pr, gt = _take_channels(pr, gt, ignore_channels=ignore_channels)

    tp = torch.sum(gt * pr)
    fp = torch.sum(pr) - tp
    fn = torch.sum(gt) - tp

    score = ((1 + beta**2) * tp + eps) / ((1 + beta**2) * tp + beta**2 * fn + fp + eps)

    return score


def accuracy(pr, gt, threshold=0.5, ignore_channels=None):
    """Calculate accuracy score between ground truth and prediction
    Args:
        pr (torch.Tensor): predicted tensor
        gt (torch.Tensor):  ground truth tensor
        eps (float): epsilon to avoid zero division
        threshold: threshold for outputs binarization
    Returns:
        float: precision score
    """
    pr = _threshold(pr, threshold=threshold)
    pr, gt = _take_channels(pr, gt, ignore_channels=ignore_channels)

    tp = torch.sum(gt == pr, dtype=pr.dtype)
    score = tp / gt.view(-1).shape[0]
    return score


def precision(pr, gt, eps=1e-7, threshold=None, ignore_channels=None):
    """Calculate precision score between ground truth and prediction
    Args:
        pr (torch.Tensor): predicted tensor
        gt (torch.Tensor):  ground truth tensor
        eps (float): epsilon to avoid zero division
        threshold: threshold for outputs binarization
    Returns:
        float: precision score
    """

    pr = _threshold(pr, threshold=threshold)
    pr, gt = _take_channels(pr, gt, ignore_channels=ignore_channels)

    tp = torch.sum(gt * pr)
    fp = torch.sum(pr) - tp

    score = (tp + eps) / (tp + fp + eps)

    return score


def recall(pr, gt, eps=1e-7, threshold=None, ignore_channels=None):
    """Calculate Recall between ground truth and prediction
    Args:
        pr (torch.Tensor): A list of predicted elements
        gt (torch.Tensor):  A list of elements that are to be predicted
        eps (float): epsilon to avoid zero division
        threshold: threshold for outputs binarization
    Returns:
        float: recall score
    """

    pr = _threshold(pr, threshold=threshold)
    pr, gt = _take_channels(pr, gt, ignore_channels=ignore_channels)

    tp = torch.sum(gt * pr)
    fn = torch.sum(gt) - tp

    score = (tp + eps) / (tp + fn + eps)

    return score

In [ ]:
class DiceLoss(Loss):
    def __init__(self, eps=1.0, beta=1.0, ignore_channels=None, **kwargs):
        super().__init__(**kwargs)
        self.eps = eps
        self.beta = beta
        self.ignore_channels = ignore_channels

    def forward(self, y_pr, y_gt):
        return 1 - f_score(
            y_pr,
            y_gt,
            beta=self.beta,
            eps=self.eps,
            threshold=None,
            ignore_channels=self.ignore_channels,
        )

In [ ]:
class FocalLoss(Loss):
    def __init__(self, alpha=1, gamma=2, class_weights=None, logits=False, reduction='mean'):
        super().__init__()
        assert reduction in ['mean', None]
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduction = reduction
        self.class_weights = class_weights if class_weights is not None else 1.

    def forward(self, y_pr, y_gt):
        bce_loss = nn.functional.binary_cross_entropy(y_pr, y_gt)

        pt = torch.exp(- bce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * bce_loss
        focal_loss = focal_loss * torch.tensor(self.class_weights).to(focal_loss.device)

        if self.reduction == 'mean':
            focal_loss = focal_loss.mean()

        return focal_loss

## CutMix

In [ ]:
def cutmix(batch, alpha=1.0, p=0):
    '''
    alpha 값을 1.0으로 설정하여 beta 분포가 uniform 분포가 되도록 함으로써,
    두 이미지를 랜덤하게 combine하는 Cutmix
    '''

    data, targets = batch

    # cutmix 확률 설정
    if np.random.random() > p:
        return data, (targets, torch.zeros_like(targets), 1.0)

    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]
    lam = np.random.beta(alpha, alpha)

    image_h, image_w = data.shape[2:]
    cx = np.random.uniform(0, image_w)
    cy = np.random.uniform(0, image_h)
    w = image_w * np.sqrt(1 - lam)
    h = image_h * np.sqrt(1 - lam)
    x0 = int(np.round(max(cx - w / 2, 0)))
    x1 = int(np.round(min(cx + w / 2, image_w)))
    y0 = int(np.round(max(cy - h / 2, 0)))
    y1 = int(np.round(min(cy + h / 2, image_h)))

    data[:, :, y0:y1, x0:x1] = shuffled_data[:, :, y0:y1, x0:x1]
    targets = (targets, shuffled_targets, lam)

    return data, targets


class CutMixCollator:
    def __init__(self, alpha, p):
        self.alpha = alpha
        self.p = p

    def __call__(self, batch):
        batch = torch.utils.data.dataloader.default_collate(batch)
        batch = cutmix(batch, self.alpha, self.p)
        return batch


In [ ]:
dice = DiceLoss()
focal = FocalLoss()
base_criterion = dice + focal

class CutMixCriterion:
    def __init__(self):
        self.criterion = base_criterion

    def __call__(self, preds, targets):
        targets1, targets2, lam = targets
        targets1 = targets1.unsqueeze(1)
        targets2 = targets2.unsqueeze(1)
        return lam * self.criterion(
            preds, targets1) + (1 - lam) * self.criterion(preds, targets2)


In [ ]:
collator = CutMixCollator(alpha=1.0, p=0.5)

## Learning Rate Scheduler

In [ ]:
class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch

        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

## Model Train

In [ ]:
valid_idx = np.arange(0,1428)
train_idx1= np.arange(1428,2856)
train_idx2= np.arange(2856,7140)
train_idx = np.concatenate((train_idx1,train_idx2 ))

print(train_idx, valid_idx)

[1428 1429 1430 ... 7137 7138 7139] [   0    1    2 ... 1425 1426 1427]


In [ ]:
def train_1epoch(model, train_loader, lossfun, optimizer, device):
    model.train()
    model.to('cuda')
    train_loss = []

    for images, masks in tqdm(train_loader):
        images = images.float().to(device)
        targets1, targets2, lam = masks # cutmix하기 위해 label split
        masks = (targets1.float().to(device), targets2.float().to(device), lam)

        optimizer.zero_grad()
        outputs = model(images)
        loss = lossfun(outputs, masks)
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())

    tr_loss = np.mean(train_loss)
    return tr_loss

def validate_1epoch(model, val_loader, lossfun, device):
    model.eval()
    model.to(device)

    val_loss = []

    with torch.no_grad():
        for images, masks in tqdm(val_loader):
            images = images.float().to(device)
            masks = masks.float().to(device)

            outputs = model(images)
            loss = lossfun(outputs, masks.unsqueeze(1))

            val_loss.append(loss.item())

    total_loss = np.mean(val_loss)
    return total_loss

In [ ]:
# Test Time Augmentation
tta_transforms = tta.Compose(
    [
        tta.HorizontalFlip(),
        tta.Rotate90(angles=[0, 90, 180]),
        #tta.Scale(scales=[1,2,4])
    ]
)

def predict(model, loader, device):

    result =[]
    with torch.no_grad():
        model.eval()

        tta_model = tta.SegmentationTTAWrapper(model, tta_transforms)
        tta_model.to(device)
        model.to(device)

        for images in tqdm(loader):
            images = images.float().to(device)

            outputs = tta_model(images)
            outputs = outputs.cpu().numpy()
            outputs = (100 * outputs).astype(np.uint8)
            masks = np.squeeze(outputs, axis=1)

            masks = masks > 35

            for i in range(len(images)):
                mask_rle = rle_encode(masks[i])
                if mask_rle == '':
                    result.append(-1)
                else:
                    result.append(mask_rle)

    return result

In [ ]:
test_ds = SatelliteDataset(csv_file='/content/drive/MyDrive/Ai/test.csv', transform=test_transform, infer=True)
test_dl = DataLoader(test_ds, batch_size=16, shuffle=False, num_workers=2)

train_ds = SatelliteDataset(csv_file='/content/drive/MyDrive/Ai/train.csv', transform=train_transform)
valid_ds = SatelliteDataset(csv_file='/content/drive/MyDrive/Ai/train.csv', transform=val_transform)

def getDataloader(train_idx, valid_idx, collator):

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_idx)

    train_dl = DataLoader(train_ds,
                          batch_size=16,
                          shuffle=False,
                          num_workers=2,
                          collate_fn = collator,
                          sampler = train_subsampler
                          )

    val_dl = DataLoader(valid_ds,
                        batch_size=16,
                        shuffle=False,
                        num_workers=2,
                        sampler = valid_subsampler
                        )


    return train_dl, val_dl

In [ ]:
def train_kfold(device):

    train_criterion = CutMixCriterion()
    val_criterion = dice

    patience = 10  # 10 epoch동안 성능 향상 없을 시, early stopping

    train_loader, val_loader = getDataloader(train_idx, valid_idx, collator=collator)

    optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=0.00001),])
    scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=10, T_mult=1,
                                          eta_max=0.001,  T_up=3, gamma=0.7)

    best_loss = 1
    best_model = None
    counter = 0

    for epoch in range(20):

        tr_loss = train_1epoch(
            model, train_loader, train_criterion, optimizer, device
        )

        val_loss = validate_1epoch(
            model, val_loader, val_criterion, device
        )

        print(f'Train Loss : [{tr_loss:.5f}] Val Loss : [{val_loss:.5f}]]')


        if scheduler is not None:
            scheduler.step()

        if best_loss > val_loss:
            best_model = model
            best_loss = val_loss
            counter=0
            # 갱신 시마다  best model 저장 -> fold별 마지막 weight이 fold별 best weight
            torch.save(model.state_dict(), f"./uppvalloss_{val_loss:.5f}.pth")
        else:
            counter+=1

        if counter > patience:
            print("Early Stopping...")
            break

        print(f'Best Val Loss : [{best_loss:.5f}]]')

    result = predict(best_model, test_dl, device)

    return result

In [ ]:
result=[]
result=train_kfold('cuda')

  5%|▌         | 19/357 [00:16<04:35,  1.23it/s]

In [ ]:
model.load_state_dict(torch.load('./upp.pth'))

## Submission

In [ ]:
submit = pd.read_csv('sample_submission.csv')
submit['mask_rle'] = result

In [ ]:
submit.to_csv('/content/drive/MyDrive/Ai/submit.csv', index=False)

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/Ai/test.csv')
submit1 = pd.read_csv('/content/drive/MyDrive/Ai/submit.csv')
submit2 = pd.read_csv('/content/drive/MyDrive/Ai/test.csv')

import matplotlib.pyplot as plt
for i in range(25000, 25050):
    test_image_path = test_data['img_path'][i]
    test_image = cv2.imread(test_image_path)
    test_mask1 = rle_decode(submit1['mask_rle'][i], (224,224))
    test_mask2 = rle_decode(submit2['mask_rle'][i], (224,224))

    plt.figure(figsize=(10,10))
    plt.subplot(131)
    plt.imshow(test_image)
    plt.axis("off")
    plt.subplot(132)
    plt.imshow(test_mask1)
    plt.axis("off")
    plt.subplot(133)
    plt.imshow(test_mask2)
    plt.axis("off")